In [22]:
#push!(LOAD_PATH, "/home/gridsan/wyao/Research/TraceFormula/Module")
push!(LOAD_PATH, "/Users/jayyao/Documents/Research/TraceFormula/Module")
using GridapEM
using Gridap
using DelimitedFiles
using KrylovKit
using LinearAlgebra

"""This part is used to define all parameters used"""
# Geometry parameters of the mesh
# Rectangular with center circle design domain
max_r = 2.0
start_r = 0.1
resol = 100.0      # Number of points per wavelength

Nri = 2
Q_list = [10, 50, 100, 1000]
Powers = zeros(Nri, length(Q_list) + 1)
Ncv = zeros(Int64, Nri, length(Q_list))
Ncv2 = zeros(Int64, Nri, length(Q_list))



λ = 1.0           # Wavelength
L = max_r * 2 + 0.5           # Width of the rectangular domain
H = max_r * 2 + 0.5           # Height of the rectangular domain
# rd = 0.7          # Radius of the design domain circle
# rt = rd + 0.2     # Radius of the target circle
dpml = 0.5        # Thickness of PML
# Characteristic length (controls the resolution, smaller the finer)
l1 = λ / resol    # Normal region
l2 = l1 / 2.0     # Design region
l3 = 2 * l1       # PML
# Physical parameters 
k = 2 * π / λ       # Wave number 
# Bloch wavevector
kb = VectorValue(0.0, 0.0)     
ω = k               # c=1
ϵ1 = 1.0            # Relative electric permittivity for material 1 (y > 0)
ϵ2 = 1.0            # Relative electric permittivity for material 2 (y < 0)
ϵ3 = 0.0            # Relative electric permittivity for potential waveguide
ϵd = 12.0           # Relative electric permittivity for design material
μ = 1.0             # Relative magnetic permeability for all materials
wg_center = [0, 0]  # Waveguide center if exist
wg_size = [0, 0]    # Waveguide size if exist
LHp=[L / 2, H / 2]  # Start of PML for x,y > 0
LHn=[L / 2, H / 2]  # Start of PML for x,y < 0

# PML parameters
R = 1e-10           # Tolerence for PML reflection 
σ1 = -3 / 4 * log(R) / dpml / √ϵ1
σ2 = -3 / 4 * log(R) / dpml / √ϵ2
σs = [σ1, σ2]
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = [0.02 * λ, 0.02 * λ]  # Filter radius
β = 80.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

# α = 1.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1

# Sum over kx
nkx = 30
nparts = nkx / 2

Bρ = true          # Matrix B depend on parameters?
ρv = 1.0

# Foundary constraint parameters
c = 0#resol^4
lw = r[1]
ls = r[1]
ηe = fηe(lw / r[1])
ηd = fηd(lw / r[1])

phys = PhysicalParameters(k, kb, ω, ϵ1, ϵ2, ϵ3, ϵd, μ, R, σs, dpml, LHp, LHn, wg_center, wg_size)


PhysicalParameters(6.283185307179586, (0.0, 0.0), 6.283185307179586, 1.0, 1.0, 0.0, 12.0, 1.0, 1.0e-10, [34.538776394910684, 34.538776394910684], 0.5, [2.25, 2.25], [2.25, 2.25], [0.0, 0.0], [0.0, 0.0])

In [15]:
for ri = 1 : Nri
    rd = (ri - 1) * (max_r - start_r) / (Nri - 1) + start_r
    Powers[ri, 1] = rd
    rt = rd + 0.2
    geo_param = CirRecGeometry(L, H, rd, rt, dpml, l1, l2, l3)
    meshfile_name = "geometry.msh"
    MeshGenerator(geo_param, meshfile_name)
    gridap = GridapFE(meshfile_name, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], [], flag_f)

    N = num_free_dofs(gridap.FE_U)
    ρ0 = ones(gridap.np)
    O_mat = MatrixOc(phys.k, phys.ϵ1; gridap)
    Neig = Int(ceil(ri / Nri * 30 * max_r))
    for qi = 1 : length(Q_list)
        α = 1.0 / (2 * Q_list[qi])
        control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bρ, ρv, c, ηe, ηd)
        
        ρf_vec = ρf_ρ0(ρ0; control, gridap)
        ρfh = FEFunction(gridap.FE_Pf, ρf_vec)
        ρth = (ρf -> Threshold(ρf; control)) ∘ ρfh

        A_mat = MatrixA(ρth; phys, control, gridap)
        B_mat = MatrixB(ρth; control, gridap)
        #@show sum(∫(ρth)gridap.dΩ_d) / sum(∫(1)gridap.dΩ_d)

        G_ii, W_raw, info = eigsolve(x -> MatrixG(x; A_mat, B_mat, O_mat), rand(ComplexF64, N), Neig, :LM; krylovdim = 50)
        Ncv[ri, qi] = num_contributing_values(G_ii, 0.99)
        Ncv2[ri, qi] = num_contributing_values(G_ii, 0.9)
        Powers[ri, qi + 1] = sum(abs.(G_ii))
    end
end

if isfile("ncv1.txt")
    run(`rm ncv1.txt`)
end
open("ncv1.txt", "w") do io
    writedlm(io, Ncv)
end

if isfile("ncv2.txt")
    run(`rm ncv2.txt`)
end
open("ncv2.txt", "w") do io
    writedlm(io, Ncv2)
end

if isfile("power.txt")
    run(`rm power.txt`)
end
open("power.txt", "w") do io
    writedlm(io, Powers)
end

Info    : No current model available: creating one
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Ci

In [13]:
ncv1 = readdlm("ncv1.txt", Int64)
ncv2 = readdlm("ncv1.txt", Int64)
power = readdlm("power.txt", Float64)

2×5 Matrix{Float64}:
 0.1   1.80467   3.01222   3.24132    3.46868
 1.0  27.4263   78.4217   94.2754   112.684

In [23]:
rd = max_r
rt = max_r + 0.2
geo_param = CirRecGeometry(L, H, rd, rt, dpml, l1, l2, l3)
meshfile_name = "geometry.msh"
MeshGenerator(geo_param, meshfile_name)
gridap = GridapFE(meshfile_name, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], [], flag_f)

N = num_free_dofs(gridap.FE_U)
ρ0 = ones(gridap.np)
O_mat = MatrixOc(phys.k, phys.ϵ1; gridap)
Neig = Int(ceil(30 * max_r))
for qi = 1 : length(Q_list)
    α = 1.0 / (2 * Q_list[qi])
    control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bρ, ρv, c, ηe, ηd)

    ρf_vec = ρf_ρ0(ρ0; control, gridap)
    ρfh = FEFunction(gridap.FE_Pf, ρf_vec)
    ρth = (ρf -> Threshold(ρf; control)) ∘ ρfh

    A_mat = MatrixA(ρth; phys, control, gridap)
    B_mat = MatrixB(ρth; control, gridap)
    #@show sum(∫(ρth)gridap.dΩ_d) / sum(∫(1)gridap.dΩ_d)

    G_ii, W_raw, info = eigsolve(x -> MatrixG(x; A_mat, B_mat, O_mat), rand(ComplexF64, N), Neig, :LM; krylovdim = Neig + 10)
    @show sum(abs.(G_ii))
end

Info    : No current model available: creating one
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Ci

In [ ]:
sum(abs.(G_ii)) = 27.564861226208418
sum(abs.(G_ii)) = 78.71712742281807
sum(abs.(G_ii)) = 95.1011696477584
sum(abs.(G_ii)) = 120.20228533910492